### DERC的行业景气度配置 by 陈策

In [ ]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm #进度条
from functools import reduce

plt.rcParams['font.family'] = ['SimHei'] #正常显示中文
plt.rcParams['axes.unicode_minus'] = False #正常显示负号

pd.options.display.max_columns = None
pd.options.display.max_rows = 10

#### 输入

#### 读取数据

In [ ]:
df = pd.read_excel('./房地产指标20210108.xlsx')


#### outlier处理

In [ ]:
def filter_outlier(x): #以3倍标准差为界，给outlier重新赋值
    x_mean = np.mean(x)
    x_std = np.std(x)
    upper_limit = x_mean + 3*x_std
    lower_limit = x_mean - 3*x_std
    
    if x > upper_limit:
        return upper_limit
    elif x < lower_limit:
        return lower_limit
    else:
        return x

#### MinMax标准化

In [ ]:
def MinMax_Standardizer(x):
    x_min = np.min(x)
    x_max = np.max(x)
    standard_x = x.apply(lambda x:(x-x_min)/(x_max-x_min))
    return standard_x

#### 数据预处理
1. 设第二行为列名
2. 修改最后两列列名
3. 仅选取某段时间数据
4. 异常值处理
5. MinMax标准化

In [ ]:
def parse_data(df, start_date = '2010-01', end_date = '2020-12'):
    df.columns = df.iloc[0].values #以首行数据为列名
    df = df.drop([0, 1]).set_index('指标名称')
    df.index.name = '日期'
    df = df[start_date:end_date].reset_index()
    df['日期'] = df['日期'].astype('str').apply(lambda x:x[:-3]) #强制转换数据格式
    for col in df.columns.tolist()[1:]:
        df[col] = df[col].astype('float')
        df[col] = filter_outlier(df[col])
        df[col] = MinMax_Standardizer(df[col])
    return df

使用z-score描述指标所处的位置高低